# Overview
In this notebook we explore the concept of Principal Component Analysis. This is a popular feature engineering technique used specifically for dimentionality reduction. In short, PCA will try to measure the importance of a feature using a measurement similar to correlation. If you are unfamiliar with correlation it is suggested you read the [notebook on correlation](Correlation.ipynb). As correlation is affected by linear scale, we will see that normalization is a prerequisite transformation for our data. If you are not famliar with normalization it is suggested you reat the [notebook on normalization]()

This notebook is broken down into the following sections:
1. Loading our data



# Step 1. Load our data

In [1]:
# Load our libraires
import pandas
import numpy

In [2]:
# Load our sample data
input_file_path = "data.csv"
delimiter = ","
df = pandas.read_csv(input_file_path, delimiter=delimiter)
df

,ACT,FinalExam,QuizAvg,TestAvg
0,33,181,95,89
1,31,169,81,89
2,21,176,65,68
3,25,181,66,90
4,29,169,89,81
5,24,103,61,57
6,25,150,81,76
7,29,147,86,76
8,36,181,98,102
9,26,163,72,70


In [3]:
df.dtypes

ACT          int64
FinalExam    int64
QuizAvg      int64
TestAvg      int64
dtype: object

# Step 2. Inspect our data
While this notebook is about doing PCA, we need to understand if we CAN do PCA using the data. Lets have a look at the data.

- We wee that the scores are different and have different units.
- We see that the columns may offer redundant information

# Step 3. ETL the data
Now that we know the problems with our data, lets transform it into a format that does not have these problems


why standardize??
https://towardsdatascience.com/a-one-stop-shop-for-principal-component-analysis-5582fb7e0a9c


## 3.1. Normalize our data
Normalization is a way to transform verctors into scalars (numbers with units into just numbers). Normalization is a handy way to convert all the measurements to a common scale. 

In our example. We need to compare the ACT (which is 1 to 36) and the average test score (which is 1 to 100). Normalization can help us do that.

https://en.wikipedia.org/wiki/Normalization_(statistics)

### 3.1.1 Choose a normalization method
There are many ways to normalize or standardiaze our data. We will not talk about all the approaches here... but there are a few and they all have different affects and implications.

### 3.2.1 Normalize using a Standard Score
We will use the **standard score** (z-score) because it is simple and well documented.

The standard score is the signed fractional number of standard deviations by which the value of an observation or data point is above or below the mean value of what is being observed or measured. Observed values above the mean have positive standard scores, while values below the mean have negative standard scores.

$$ z = \frac{(x-\mu)}{\sigma} $$

The higher or lower the score, the farther from the mean, and thus the more exceptional a given score is when compared to the sample. This is how we will compare our scores!!

If we look at a sample with a normally distributed random variable, we will see the implications of the various scores:

![image](Normalization_method_comparison.png)

First we will calculate some statistics for our columns. We will use these statistics to normalize our data.

In [4]:
# Calculate the mean of each column in the data frame
df.mean()

ACT           27.714286
FinalExam    156.428571
QuizAvg       77.071429
TestAvg       76.428571
dtype: float64

In [5]:
# Calculate the standard deviation of each column in the data frame
df.std()

ACT           4.158931
FinalExam    26.117234
QuizAvg      13.747527
TestAvg      13.195071
dtype: float64

In [6]:
# Calculate the standard score of each element in our data frame (on a per-column basis)
normalized_df = ((df - df.mean())/df.std())
normalized_df

,ACT,FinalExam,QuizAvg,TestAvg
0,1.270931,0.940813,1.304131,0.952737
1,0.790038,0.481346,0.285766,0.952737
2,-1.614426,0.749368,-0.878080,-0.638767
3,-0.652640,0.940813,-0.805340,1.028523
4,0.309145,0.481346,0.867689,0.346450
5,-0.893087,-2.045721,-1.169041,-1.472411
6,-0.652640,-0.246143,0.285766,-0.032480
7,0.309145,-0.361010,0.649467,-0.032480
8,1.992270,0.940813,1.522352,1.937953
9,-0.412194,0.251613,-0.368898,-0.487195


In [ ]:
normalized_df.dtypes

# Step 4. Covariance and Correlation

## 4.1. Quick stats recap:

In this step we calculate some statistics for our data frame. Lets have a quick recap.

$$ \mu = \frac{\sum(x_i)}{n} $$

$$  \sigma^2 = \frac{\sum(x_i-\mu)^2}{n} $$

$$ Cov(X,Y) = \frac{\sum(x_i-\mu_X)(y_i - \mu_Y)}{n} $$

$$ \rho_XY = \frac{Cov(X,Y)}{\sigma_X \sigma_Y} $$

Given the nature of the z-score, we see that multiplying two z-scores of two columns would yield the correlation coefficient. 
Using matrices, multiplying a column by its transpose, $Z^TZ$, produces the correlation coefficient matrix. 

$$ z = \frac{x-\mu}{\sigma} $$

## 4.2 Calculate Covariance and Correlation by hand

In [7]:
# Get the number of rows
n = normalized_df.shape[0] - 1

# Take the transpose of our matrix
normalized_transposed_df = normalized_df.T

# Calculate correlation by taking the dot product of our transpose and regular matrix
#     Remember that the z-score includes the sigma
correlation_df = normalized_transposed_df.dot(normalized_df) / n
correlation_df

,ACT,FinalExam,QuizAvg,TestAvg
ACT,1.000000,0.336186,0.827803,0.710275
FinalExam,0.336186,1.000000,0.499949,0.795847
QuizAvg,0.827803,0.499949,1.000000,0.748696
TestAvg,0.710275,0.795847,0.748696,1.000000


## 4.3. Calculate Correlation using built in function

In [8]:
# Take the transpose of our matrix
normalized_transposed_df = normalized_df.T

# Compute the correlation matrix (a numpy ndarray)
correlation_nda = numpy.corrcoef(normalized_transposed_df)
correlation_nda

array([[1.        , 0.33618637, 0.82780343, 0.7102748 ],
       [0.33618637, 1.        , 0.49994932, 0.79584684],
       [0.82780343, 0.49994932, 1.        , 0.7486962 ],
       [0.7102748 , 0.79584684, 0.7486962 , 1.        ]])

In [ ]:
# Convert the numpy array into a pandas data frame
correlation_df = pandas.DataFrame(correlation_nda, columns=df.columns, index=df.columns)
correlation_df

We see that QuizAvg and ACT have the highest correlation meaning that they are redundant features. There is only a small benefit to including both because one of them give us most of the information we need to explain the variance.

# Step 5. Decompose correlation matrix into eigenvalues and eigenvectors

In German, “eigen” means “specific of” or “characteristic of”. Eigenvectors and eigenvalues are also referred to as characteristic vectors and latent roots or characteristic equation.The set of eigenvalues of a matrix is also called its spectrum.

*Encyclopedia of Measurement and Statistics* ~ Abdi

## Fundamental Theory of matrix eigenvectors and eigenvalues
A (non-vero) vector $v$ of dimension N is an *eigenvector* of a square NxN matrix A if it satisfies the linear equation

$$Av = \lambda v $$

where $\lambda$ is a scalar, termed the *eigenvlue*, which corespondes to the eigenvector $v$.

There are multiple possible eigenvectors for the NxN matrix A. The spectral theroy allows us to capture those.

## Spectral Theory

Let A be square NxN matrix of eigenvectors. Let Q be a square NxN matrix who's i'th column, denoted $q_i$ is an eigenvector of A.

$$ A = Q \Lambda Q^{-1} $$

As this is a special case (square matrix) we can say that $Q$ is orthogonal and thus:

$$ A = Q \Lambda Q^T $$

We can apply the spectral theory to decompose our square correlation matrix into its canonical form: two square matrices consisting of eigenvalues $\Lambda$ and eigenvectors $Q$ respectively.

https://en.wikipedia.org/wiki/Eigendecomposition_of_a_matrix

We really do not need to know how this works (like the theory and all). Lets focus on the application.

We can use a builtin numpy function to do the decomposition:
https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html

In [9]:
# Do the decomposition
eigen_values_nda, right_eigen_vectors_nda = numpy.linalg.eig(correlation_nda)

In [10]:
# Look at our lamndas
eigen_values_df = pandas.DataFrame(eigen_values_nda, index=df.columns, columns=["eigenvalues"])
eigen_values_df

,eigenvalues
ACT,2.978867
FinalExam,0.762946
QuizAvg,0.083230
TestAvg,0.174956


In [11]:
# Look at our Qs
#    Note: The term right vector just refers to the orientation of rows vs columns
#          This is part of the math that we just do not need to worry about
right_eigen_vectors_df = pandas.DataFrame(right_eigen_vectors_nda, columns=df.columns, index=df.columns)
right_eigen_vectors_df

,ACT,FinalExam,QuizAvg,TestAvg
ACT,0.488657,0.548902,-0.485805,0.473196
FinalExam,0.435042,-0.735297,-0.506504,-0.116321
QuizAvg,0.521538,0.330721,0.145362,-0.772976
TestAvg,0.547678,-0.220610,0.697364,0.406280


# Step 6. Transform normalized data using eigenvectors
This explains the reason why we multiply the normalized data Z with P. The fact that the covariance of ZP is the diagonal matrix indicates that there is no linear relationship between variables other than the one with itself. In other words, there is no relationship among the new variables we created and the new variables are independent of one another. Therefore, we have successfully come up with a new set of dataset (a transformed dataset) which does not have any correlation between them and we can select variables based on the variance, which is represented by the eigenvalues.

The important thing here is that the total variance of the original dataset is the same as that of the transformed dataset. That is to say, the sum of the entries of the sum of the entries of the diagonal matrix D is the sum of the variance of the Z scores. With the covariance matrix of ZP being D, each vector of ZP represents the variance of the transformed data and by choosing the vectors that correspond to the highest eigenvalues, the variance is maximized. Thus, by selecting the vectors that correspond to highest eigenvalues, we are selecting the new variables that have high fraction of the variance of the transformed dataset divided by the total variance of the original dataset.

https://towardsdatascience.com/principal-component-analysis-ceb42ed04d77

In [12]:
normalized_transformed_df = pandas.DataFrame(numpy.dot(normalized_df, right_eigen_vectors_df), columns=df.columns)
normalized_transformed_df

,ACT,FinalExam,QuizAvg,TestAvg
0,2.232289,0.226959,-0.239975,-0.129021
1,1.266294,-0.035954,0.078336,0.484040
2,-1.270685,-1.586650,-0.168354,-0.431890
3,0.233659,-1.543257,0.440720,0.622113
4,1.002747,0.026290,-0.026258,-0.439651
5,-2.742494,0.952199,0.273291,0.120785
6,-0.294750,-0.075573,0.460618,-0.514281
7,0.314946,0.657097,0.104426,-0.326939
8,3.238168,0.477725,0.128371,0.443908
9,-0.551179,-0.425786,-0.320573,-0.137105


# Step 7. Sort our eigenvalues and eigenvectors


In [13]:
# Recall our eigenvalues
eigen_values_df

,eigenvalues
ACT,2.978867
FinalExam,0.762946
QuizAvg,0.083230
TestAvg,0.174956


In [15]:
# Create a mapping of the eigenvalue's value to its position
# We will use this mapping to reorder the eigenvector later
mapping = {}
for x in range(0, eigen_values_nda.size):
    mapping[eigen_values_nda[x]] = x
mapping

{2.9788674250300993: 0,
 0.7629463524576332: 1,
 0.08322983246250416: 2,
 0.1749563900497669: 3}

In [23]:
# Sort the eigenvalues from largest to smallest
sorted_eigen_values_nda = eigen_values_nda.astype(float)
sorted_eigen_values_nda.sort()
sorted_eigen_values_nda = sorted_eigen_values_nda[::-1]
sorted_eigen_values_nda

array([2.97886743, 0.76294635, 0.17495639, 0.08322983])

In [24]:
# Recall the original column name order
original_order_column_names = df.columns.tolist()
original_order_column_names

['ACT', 'FinalExam', 'QuizAvg', 'TestAvg']

In [25]:
# Reorder the columns in the dataframe to match the eigenvalue order
sorted_column_names = []
for eigenvalue in sorted_eigen_values_nda:
    original_order = mapping[eigenvalue]
    column_name = original_order_column_names[original_order]
    sorted_column_names.append(column_name)
sorted_column_names

['ACT', 'FinalExam', 'TestAvg', 'QuizAvg']

In [26]:
# Create a new DF with the eigenvectors in sorted order
sorted_right_eigen_vectors_df = right_eigen_vectors_df[sorted_column_names]
sorted_right_eigen_vectors_df

,ACT,FinalExam,TestAvg,QuizAvg
ACT,0.488657,0.548902,0.473196,-0.485805
FinalExam,0.435042,-0.735297,-0.116321,-0.506504
QuizAvg,0.521538,0.330721,-0.772976,0.145362
TestAvg,0.547678,-0.220610,0.406280,0.697364


# Step 8. Select Principal Components

There are a few methods for doing this:
1. Guess/Trial and Error

    Basically... just pick the ones with the highest eigenvalues and then run the model. Rinse and repeat until you find the sweet spot.
    

2. Arbitrarily set a variance threshold

    Pick a threshold, and add features until you hit that threshold. (For example, if you want to explain 80% of the total variability possibly explained by your model, add features with the largest explained proportion of variance until your proportion of variance explained hits or exceeds 80%.)
    

3. Scree Plot & Find the Elbow

    This is closely related to Method 2. Calculate the proportion of variance explained for each feature, sort features by proportion of variance explained and plot the cumulative proportion of variance explained as you keep more features. (This plot is called a scree plot, shown below.) One can pick how many features to include by identifying the point where adding a new feature has a significant drop in variance explained relative to the previous feature, and choosing features up until that point. (I call this the “find the elbow” method, as looking at the “bend” or “elbow” in the scree plot determines where the biggest drop in proportion of variance explained occurs.)

https://towardsdatascience.com/a-one-stop-shop-for-principal-component-analysis-5582fb7e0a9c

# 8.2. Method 2: Variance Threshold

In [27]:
# Review our eigenvalues
sorted_eigen_values_nda

array([2.97886743, 0.76294635, 0.17495639, 0.08322983])

In [28]:
# Calculate the total variance explained by the eigenvalues
total_variance_explained_by_eigenvalues = sum(sorted_eigen_values_nda)
total_variance_explained_by_eigenvalues

4.0000000000000036

In [29]:
# Determine the variance explained by each principal component
variance_map = {}
n = sorted_eigen_values_nda.size
for x in range(0,n):
    feature_name = sorted_column_names[x]
    feature_eigenvalue = sorted_eigen_values_nda[x]
    feature_percentage = feature_eigenvalue / total_variance_explained_by_eigenvalues
    variance_map[feature_name] = feature_percentage
    print("The feature {0} accounted for {1}% of the total variance.".format(feature_name, feature_percentage*100))

The feature ACT accounted for 74.47168562575241% of the total variance.
The feature FinalExam accounted for 19.073658811440815% of the total variance.
The feature TestAvg accounted for 4.3739097512441685% of the total variance.
The feature QuizAvg accounted for 2.0807458115626023% of the total variance.


In [30]:
# Set our variance threshold to 90%
variance_threshold = 0.90

In [33]:
# Select our PCAs
feature_set = []
variance_explained_by_feature_set = 0
for x in range(0,sorted_eigen_values_nda.size):
    if variance_explained_by_feature_set >= variance_threshold:
        break
    feature_name = sorted_column_names[x]
    feature_set.append(feature_name)
    feature_percentage = variance_map[feature_name]
    variance_explained_by_feature_set += feature_percentage
print("Our feature set is:")
print(feature_set)
print("The percentage of variance explained is:")
print(variance_explained_by_feature_set)    

Our feature set is:
['ACT', 'FinalExam']
The percentage of variance explained is:
0.9354534443719322


# 8.3. Method 3: Scree Plot & Find the Elbow
This is closely related to Method 2. Calculate the proportion of variance explained for each feature, sort features by proportion of variance explained and plot the cumulative proportion of variance explained as you keep more features. (This plot is called a scree plot, shown below.) One can pick how many features to include by identifying the point where adding a new feature has a significant drop in variance explained relative to the previous feature, and choosing features up until that point. (I call this the “find the elbow” method, as looking at the “bend” or “elbow” in the scree plot determines where the biggest drop in proportion of variance explained occurs.)

We need to write some code to render the Scree Plot and allow us to find the elbow visually. There are methods for finding the elbow mathematically, we will not cover those methods here.